<a href="https://colab.research.google.com/github/Maximo-Rulli/dynamic-steps-dlm/blob/main/blocks-entropy-test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyzing the entropy of Diffusion blocks by DLMs

### Essential imports

In [41]:
#Essential imports
import torch
import numpy as np
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModel

#Repository's functions
from MMaDA.models import MMadaModelLM
from MMaDA.generate import generate, custom_generate
import MMaDA.generate as gen
import importlib
importlib.reload(gen)

<module 'MMaDA.generate' from '/home/maxrul/dev/dynamic-steps-dlm/MMaDA/generate.py'>

### Tokenizer and model loading

In [2]:
device = 'cuda'
model = MMadaModelLM.from_pretrained("Gen-Verse/MMaDA-8B-Base", trust_remote_code=True, torch_dtype=torch.bfloat16).to(device).eval()
tokenizer = AutoTokenizer.from_pretrained("Gen-Verse/MMaDA-8B-Base", trust_remote_code=True)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
You are using a model of type llada to instantiate a model of type mmada. This is not supported for all configurations of models and can yield errors.


Initializing MMadaModelLM with config: MMadaConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "Gen-Verse/MMaDA-8B-Base",
  "activation_type": "silu",
  "alibi": false,
  "alibi_bias_max": 8.0,
  "architectures": [
    "LLaDAModelLM"
  ],
  "attention_dropout": 0.0,
  "attention_layer_norm": false,
  "attention_layer_norm_with_affine": true,
  "auto_map": {
    "AutoConfig": "Gen-Verse/MMaDA-8B-Base--configuration_llada.LLaDAConfig",
    "AutoModel": "Gen-Verse/MMaDA-8B-Base--modeling_llada.LLaDAModelLM",
    "AutoModelForCausalLM": "Gen-Verse/MMaDA-8B-Base--modeling_llada.LLaDAModelLM"
  },
  "bias_for_layer_norm": false,
  "block_group_size": 1,
  "block_type": "llama",
  "codebook_size": 8192,
  "d_model": 4096,
  "embedding_dropout": 0.0,
  "embedding_size": 134656,
  "eos_token_id": 126081,
  "flash_attention": false,
  "include_bias": false,
  "include_qkv_bias": false,
  "init_cutoff_factor": null,
  "init_device": "meta",
  "init_fn": "mitchell",
  "init_std": 

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  9.71it/s]


### Load tokenizer chat template

In [37]:
tokenizer.chat_template = "{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{{ '<|start_header_id|>assistant<|end_header_id|>\n' }}"

### Set prompt and tokenize it

In [120]:
prompt = "If I have 2 friends and 6 apples, how many apples does each one recieve?"
m = [{"role": "user", "content": prompt},]
prompt = tokenizer.apply_chat_template(m, add_generation_prompt=True, tokenize=False)
input_ids = tokenizer(text=prompt, return_tensors="pt", padding=True, padding_side="left")['input_ids']
input_ids = input_ids.detach().clone().to(device)

In [121]:
input_ids

tensor([[126080, 126346,   3840, 126347,    198,   2531,    331,    561,    220,
             17,   4569,    301,    220,     21,  32993,     11,   1099,   1494,
          32993,   1543,   1671,    810,   1168,   2925,     30, 126348, 126346,
            598,  10450, 126347,    198]], device='cuda:0')

### Run inference on the model

In [122]:
importlib.reload(gen)

<module 'MMaDA.generate' from '/home/maxrul/dev/dynamic-steps-dlm/MMaDA/generate.py'>

With more than 47 steps, and length, the answer gets considerably shorter and concise. To be researched!!!

prompt: "If I have 2 friends and 6 apples, how many apples does each one recieve?"

steps<=47:
answer: "Each friend receives 3 apples."

steps>47:
answer: "3"

In [146]:
length = 32
out = gen.custom_generate(model, input_ids, steps=length, gen_length=length, block_length=length//2, temperature=0, cfg_scale=0., remasking='low_confidence')
print(tokenizer.batch_decode(out[:, input_ids.shape[1]:], skip_special_tokens=True))

tensor([[126080, 126346,   3840, 126347,    198,   2531,    331,    561,    220,
             17,   4569,    301,    220,     21,  32993,     11,   1099,   1494,
          32993,   1543,   1671,    810,   1168,   2925,     30, 126348, 126346,
            598,  10450, 126347,    198,  11934,   2684,  21740,    220,     18,
          32993,     13, 126081, 126081, 126081, 126081, 126081, 126081, 126081,
         126081, 126081, 126081, 126081, 126081, 126081, 126081, 126081, 126081,
         126081, 126081, 126081, 126081, 126081, 126081, 126081, 126081, 126081]],
       device='cuda:0')
['Each friend receives 3 apples.']


In [33]:
generate(model, input_ids, steps=128, gen_length=128, block_length=128, temperature=1, cfg_scale=0., remasking='low_confidence')

tensor([[126080, 126346,   3840, 126347,    198,   3808,    268,   6866,    698,
          22391,    301,    654,    268,   6866,    698,   3484,     11, 109466,
           1227,   2537,  24036,   5223,   3361,   1750,     13, 126348, 126346,
            598,  10450, 126347,    198,   9885, 126081, 126081, 126081, 126081,
         126081, 126081, 126081, 126081, 126081, 126081, 126081, 126081, 126081,
         126081, 126081, 126081, 126081, 126081, 126081, 126081, 126081, 126081,
         126081, 126081, 126081, 126081, 126081, 126081, 126081, 126081, 126081,
         126081, 126081, 126081, 126081, 126081, 126081, 126081, 126081, 126081,
         126081, 126081, 126081, 126081, 126081, 126081, 126081, 126081, 126081,
         126081, 126081, 126081, 126081, 126081, 126081, 126081, 126081, 126081,
         126081, 126081, 126081, 126081, 126081, 126081, 126081, 126081, 126081,
         126081, 126081, 126081, 126081, 126081, 126081, 126081, 126081, 126081,
         126081, 126081, 126